In [10]:
import cv2
import numpy as np
import os
import pandas as pd

# 이미지 폴더 (사진 불러올 곳)
image_folder = r"C:/Users/USER/Downloads/Picture_250531/outlier/"

# 결과 저장 경로 (바탕화면 1차 프로젝트 폴더)
output_csv = r"C:/Users/USER/OneDrive/바탕 화면/1차 프로젝트/project/drain_surface_results.csv"

results = []

# 이미지 파일 리스트 (a-z 정렬)
image_files = sorted([f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

for filename in image_files:
    img_path = os.path.join(image_folder, filename)
    img = cv2.imread(img_path)
    if img is None:
        print(f"이미지 로드 실패: {filename}")
        continue

    print(f"처리 중: {filename}")

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=50, maxLineGap=10)

    grid_mask = np.zeros_like(gray)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(grid_mask, (x1, y1), (x2, y2), 255, 2)

    holes_mask = cv2.bitwise_not(grid_mask)
    _, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)
    masked_binary = cv2.bitwise_and(binary, binary, mask=holes_mask)

    contours, _ = cv2.findContours(masked_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    blocked_holes = 0
    total_holes = len(contours)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < 100:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h
        if not (0.5 < aspect_ratio < 2.0):
            continue
        blocked_holes += 1

    surface_ratio = blocked_holes / total_holes if total_holes > 0 else 0

    if surface_ratio >= 0.5:
        label = "Heavy"
    else:
        label = "Check Internal"

    print(f"파일: {filename} | 표면 막힘 비율: {surface_ratio:.2f} | 레이블: {label}")

    results.append({
        "filename": filename,
        "surface_ratio": round(surface_ratio, 3),
        "label": label
    })

# 결과를 CSV로 저장
df = pd.DataFrame(results)
df.to_csv(output_csv, index=False)
print(f"\n✅ 모든 이미지 처리 완료! 결과 CSV 저장 위치 → {output_csv}")

처리 중: BLOCKED.jpg
파일: BLOCKED.jpg | 표면 막힘 비율: 0.04 | 레이블: Check Internal
처리 중: CLEAR.jpg
파일: CLEAR.jpg | 표면 막힘 비율: 0.01 | 레이블: Check Internal
처리 중: WARNING.jpg
파일: WARNING.jpg | 표면 막힘 비율: 0.03 | 레이블: Check Internal

✅ 모든 이미지 처리 완료! 결과 CSV 저장 위치 → C:/Users/USER/OneDrive/바탕 화면/1차 프로젝트/project/drain_surface_results.csv
